In [1]:
from __future__ import division
from __future__ import print_function

import os, sys
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import random
import config
import torch.optim as optim
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
# NEURAL NETWORK MODULES/LAYERS
from dataset import Dataset
# METRICS CLASS FOR EVALUATION
from metrics import Metrics
# CONFIG PARSER
from config import parse_args
# TRAIN AND TEST HELPER FUNCTIONS
from trainer import Trainer
import glob
from tqdm import tqdm
from PIL import Image
import torchvision
from torchvision import models
from torch.utils.tensorboard import SummaryWriter

import model_vgg16
# import model_vgg19
# import model_resnet50
# import model_densenet121
# import model_inceptionv3

# MAIN BLOCK
# def main():
# global args
# args = parse_args()
# global logger


/home/mshaikh2/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mshaikh2/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mshaikh2/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/hom

In [2]:
torch.__version__

'1.2.0'

In [3]:
class config():
    def __init__(self):
        self.data='./data'
        self.save='checkpoints/'
        self.expname='vgg16_holistic'
        self.expno=0
        self.pretrained_model='vgg16'
        self.num_classes=16
        # training arguments
        self.epochs=100000
        self.batchsize=64
        self.lr=0.0001
        self.wd=1e-4
        self.sparse=False
        self.optim='adam'
        # miscellaneous options
        self.seed=123
        self.cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.cuda else "cpu")
        self.pretrained_holistic=0

#         cuda_parser = parser.add_mutually_exclusive_group(required=False)
#         cuda', dest='cuda', action='store_true')
#         cuda_parser.add_argument('--no-cuda', dest='cuda', action='store_false')
#         parser.set_defaults(cuda=True)
        

In [4]:
args = config()
writer = SummaryWriter(comment = args.expname)
args.batchsize

64

In [5]:

torch.manual_seed(args.seed)
random.seed(args.seed)

if args.cuda:
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.benchmark = True

if not os.path.exists(args.save):
    os.makedirs(args.save)



In [6]:
# initialize model, criterion/loss_function, optimizer
pretrained_vgg16 = models.vgg16(pretrained=True)

# Freeze training for all layers
# for child in pretrained_vgg16.children():
#     for param in child.parameters():
#         param.requires_grad = False
start_point = 0

if args.pretrained_holistic == 0:
    if os.path.exists('./checkpoints/vgg16_holistic.pt'):
        model = model_vgg16.DocClassificationHolistic(args, pretrained_vgg16)
        checkpoint = torch.load('./checkpoints/vgg16_holistic.pt')
        model.load_state_dict(checkpoint['model'])
        start_point = checkpoint['epoch']
    else:
        model = model_vgg16.DocClassificationHolistic(args, pretrained_vgg16)
elif args.pretrained_holistic == 1:
    if os.path.exists('./checkpoints/vgg16_ensemble.pt'):
        pretrained_orig_vgg16 = model_vgg16.DocClassificationHolistic(args, pretrained_vgg16)
        pretrained_holistic = model_vgg16.DocClassificationHolistic(args, pretrained_orig_vgg16.pretrained_model)
        checkpoint = torch.load('./checkpoints/vgg16_holistic.pt')
        pretrained_holistic.load_state_dict(checkpoint['model'])
        
        model = model_vgg16.DocClassificationRest(args, pretrained_orig_vgg16, pretrained_holistic)
        checkpoint = torch.load('./checkpoints/vgg16_ensemble.pt')
        model.load_state_dict(checkpoint['model'])
        start_point = checkpoint['epoch']
    else:
        pretrained_orig_vgg16 = model_vgg16.DocClassificationHolistic(args, pretrained_vgg16)
        pretrained_holistic = model_vgg16.DocClassificationHolistic(args, pretrained_orig_vgg16.pretrained_model)
        checkpoint = torch.load('./checkpoints/vgg16_holistic.pt')
        pretrained_holistic.load_state_dict(checkpoint['model'])

        model = model_vgg16.DocClassificationRest(args, pretrained_orig_vgg16, pretrained_holistic)

criterion = nn.CrossEntropyLoss(reduction='mean')

parameters = filter(lambda p: p.requires_grad, model.parameters())

if args.cuda:
    model.cuda(), criterion.cuda()

if args.optim=='adam':
    optimizer   = optim.Adam(parameters, lr=args.lr, weight_decay=args.wd)
elif args.optim=='adagrad':
    optimizer   = optim.Adagrad(parameters, lr=args.lr, weight_decay=args.wd)
elif args.optim=='sgd':
    optimizer   = optim.SGD(parameters, lr=args.lr, weight_decay=args.wd)
elif args.optim == 'adadelta':
    optimizer = optim.Adadelta(parameters, lr=args.lr, weight_decay=args.wd)
metrics = Metrics(args.num_classes)

# create trainer object for training and testing
trainer = Trainer(args, model, criterion, optimizer)

In [7]:
# for child in pretrained_vgg16.children():
#     for param in child.parameters():
#         param.requires_grad = True
#         print(param)
# for parameter in model.parameters():
#     print(parameter)

In [8]:
train_dir = glob.glob(os.path.join(args.data,'train/holistic/*.jpg'))
dev_dir = glob.glob(os.path.join(args.data,'val/holistic/*.jpg'))
test_dir = glob.glob(os.path.join(args.data,'test/holistic/*.jpg'))

print(len(train_dir), len(dev_dir), len(test_dir))

# 'data/labels/train.csv'
train_dataset = Dataset('data/labels/train.csv','train', train_dir, model_type=args.pretrained_model)
dev_dataset = Dataset('data/labels/val.csv','val',dev_dir, model_type=args.pretrained_model)
test_dataset = Dataset('data/labels/test.csv','test', test_dir, model_type=args.pretrained_model)


print('==> Size of train data   : %d ' % len(train_dataset))
print('==> Size of val data   : %d ' % len(dev_dataset))
print('==> Size of test data   : %d ' % len(test_dataset))

train_idx = list(np.arange(len(train_dataset)))
dev_idx = list(np.arange(len(dev_dataset)))
test_idx = list(np.arange(len(test_dataset)))

best = float('inf')
columns = ['ExpName','ExpNo', 'Epoch', 'Loss','Accuracy']
results = []
early_stop_count = 0

80000 16000 15999
==> Size of train data   : 80000 
==> Size of val data   : 16000 
==> Size of test data   : 15999 


In [9]:
bh,h,f,l,r,l = train_dataset[np.random.randint(55000,60000,64)]
batch_holistic = torch.unsqueeze(bh, dim=1).repeat(1,3,1,1).float().cuda()
writer.add_graph(model, input_to_model=batch_holistic)
# writer.flush()
bh = np.array(bh.detach().cpu())
bh.shape

(64, 224, 224)

In [10]:
from matplotlib import pyplot as plt
def plot_data(x,ax):
#     x = x.reshape((size,size))
    
    ax.imshow(x, cmap='gray')
#     if y is not None:
#         ax.scatter(y[0::2] , y[1::2] , marker='x', s=10)
def plot_images(x,batch_size=64):
    fig = plt.figure(figsize=(50,50))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.01, wspace=0.01)
    for i in range(batch_size):
        ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    #     print(y['classes'][i])
    #     print(np.argmax(c[i]))
        plot_data(x[i], ax)
plot_images(bh)
print(l)
# display(plot_images(np.squeeze(seg[0],axis=-1)))

tensor([11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
        11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
        11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
        11, 11, 11, 11, 11, 11, 11, 11, 11, 11])


In [ ]:


for epoch in tqdm(range(start_point, args.epochs), desc= 'training'):

    train_loss = 0.0
    dev_loss = 0.0
    test_loss = 0.0

    train_predictions = []
    train_labels = []

    dev_predictions = []
    dev_labels = []

    test_predictions = []
    test_labels = []

    

    batch_train_data = list(np.random.randint(0, len(train_idx), args.batchsize))
    batch_dev_data = list(np.random.randint(0, len(dev_idx), args.batchsize))
    batch_test_data = list(np.random.randint(0, len(test_idx), args.batchsize))
    
#     for batch in tqdm(batch_train_data, desc='Training batches..'):
    train_batch_holistic, \
            train_batch_header, \
            train_batch_footer, \
            train_batch_left_body, \
            train_batch_right_body, \
            train_batch_labels = train_dataset[batch_train_data]
    
    if args.pretrained_holistic == 0:
                _ = trainer.train_holistic(train_batch_holistic, train_batch_labels)
    elif args.pretrained_holistic == 1:
        _ = trainer.train_rest(train_batch_holistic, \
                                train_batch_header, \
                                train_batch_footer, \
                                train_batch_left_body, \
                                train_batch_right_body, \
                                train_batch_labels)

#     for batch in tqdm(batch_train_data, desc='Training batches..'):
    train_batch_holistic, \
            train_batch_header, \
            train_batch_footer, \
            train_batch_left_body, \
            train_batch_right_body, \
            train_batch_labels = train_dataset[batch_train_data]
    
    if args.pretrained_holistic == 0:
                train_batch_loss, train_batch_predictions, train_batch_labels = trainer.test_holistic(train_batch_holistic, train_batch_labels)
    elif args.pretrained_holistic == 1:
        train_batch_loss, train_batch_predictions, train_batch_labels = trainer.test_rest(train_batch_holistic, \
                                                                                    train_batch_header, \
                                                                                    train_batch_footer, \
                                                                                    train_batch_left_body, \
                                                                                    train_batch_right_body, \
                                                                                    train_batch_labels)
#         train_predictions.append(train_batch_predictions)
#         train_labels.append(train_batch_labels)
#         train_loss = train_loss + train_batch_loss

    train_accuracy = metrics.accuracy(train_batch_predictions, train_batch_labels)

#     for batch in tqdm(batch_dev_data, desc='Dev batches..'):
    dev_batch_holistic, \
            dev_batch_header, \
            dev_batch_footer, \
            dev_batch_left_body, \
            dev_batch_right_body, \
            dev_batch_labels = dev_dataset[batch_dev_data]

    if args.pretrained_holistic == 0:
                dev_batch_loss, dev_batch_predictions, dev_batch_labels = trainer.test_holistic(dev_batch_holistic, dev_batch_labels)
    elif args.pretrained_holistic == 1:
        dev_batch_loss, dev_batch_predictions, dev_batch_labels = trainer.test_rest(dev_batch_holistic, \
                                                                                dev_batch_header, \
                                                                                dev_batch_footer, \
                                                                                dev_batch_left_body, \
                                                                                dev_batch_right_body, \
                                                                                dev_batch_labels)
#         dev_predictions.append(dev_batch_predictions)
#         dev_labels.append(dev_batch_labels)
#         dev_loss = dev_loss + dev_batch_loss

    dev_accuracy = metrics.accuracy(dev_batch_predictions, dev_batch_labels)

#     for batch in tqdm(batch_test_data, desc='Test batches..'):
    test_batch_holistic, \
            test_batch_header, \
            test_batch_footer, \
            test_batch_left_body, \
            test_batch_right_body, \
            test_batch_labels = test_dataset[batch_test_data]

    if args.pretrained_holistic == 0:
                test_batch_loss, test_batch_predictions, test_batch_labels = trainer.test_holistic(test_batch_holistic, test_batch_labels)
    elif args.pretrained_holistic == 1:
        test_batch_loss, test_batch_predictions, test_batch_labels = trainer.test_rest(test_batch_holistic, \
                                                                                test_batch_header, \
                                                                                test_batch_footer, \
                                                                                test_batch_left_body, \
                                                                                test_batch_right_body, \
                                                                                test_batch_labels)

#         test_predictions.append(test_batch_predictions)
#         test_labels.append(test_batch_labels)
#         test_loss = test_loss + test_batch_loss

    test_accuracy = metrics.accuracy(test_batch_predictions, test_batch_labels)

    writer.add_scalar('Loss/train',train_batch_loss.detach().cpu(), epoch)
    writer.add_scalar('Loss/test', test_batch_loss.detach().cpu(), epoch)
    writer.add_scalar('Loss/val', dev_batch_loss.detach().cpu(), epoch)
    writer.add_scalar('Accuracy/val', dev_accuracy, epoch)
    writer.add_scalar('Accuracy/train', train_accuracy, epoch)
    writer.add_scalar('Accuracy/test', test_accuracy, epoch)
    
#     print('Epoch {:7d}, Train Loss: {:.2f}, Train Accuracy: {:.2f},' \
#           'Dev Loss: {:.2f}, Dev Accuracy: {:.2f}, Test Loss: {:.2f},' \
#           'Test Accuracy: {:.2f}'.format(epoch + 1 \
#                                      , train_batch_loss.detach().cpu() \
#                                      , train_accuracy \
#                                      , dev_batch_loss.detach().cpu() \
#                                      , dev_accuracy \
#                                      , test_batch_loss.detach().cpu() \
#                                      , test_accuracy), end = '\r')
    
#     results.append((args.expname, \
#                     args.expno, \
#                     epoch+1, \
#                     test_batch_loss, \
#                     test_accuracy))

    if best > test_batch_loss:
        best = test_batch_loss
        checkpoint = {'model': trainer.model.state_dict(), 'optim': trainer.optimizer,
                      'loss': test_batch_loss, 'accuracy': test_accuracy,
                      'args': args, 'epoch': epoch }
#         logger.debug('==> New optimum found, checkpointing everything now...')
        torch.save(checkpoint, '%s.pt' % os.path.join(args.save, args.expname))
        #np.savetxt("test_pred.csv", test_pred.numpy(), delimiter=",")
        early_stop_count = 0
    else:
        early_stop_count = early_stop_count + 1

        if early_stop_count == 1250:
            print('Early Stopping')
            print('Epoch {:7d}, Train Loss: {:.2f}, Train Accuracy: {:.2f},' \
          'Dev Loss: {:.2f}, Dev Accuracy: {:.2f}, Test Loss: {:.2f},' \
          'Test Accuracy: {:.2f}'.format(epoch + 1 \
                                     , train_batch_loss.detach().cpu() \
                                     , train_accuracy \
                                     , dev_batch_loss.detach().cpu() \
                                     , dev_accuracy \
                                     , test_batch_loss.detach().cpu() \
                                     , test_accuracy))
    
            break



training:   5%|▌         | 5151/100000 [2:40:06<50:55:35,  1.93s/it]

In [ ]:
# bh = train_dataset[np.random.randint(0,10000,64)][0]

In [ ]:
# bh = train_dataset[np.random.randint(0,10000,64)][0]
# batch_holistic = torch.unsqueeze(bh, dim=1).repeat(1,3,1,1).float()

In [ ]:
# import os,numpy as np

In [ ]:
# l = np.array(sorted(os.listdir('data/test/holistic/')))
# _,cts = np.unique(l,return_counts=True)
# cts[cts>1]

In [ ]:
# import pandas as pd

In [ ]:
# pd.read_csv('data/labels/test.csv',sep=' ')
# np.unique(l,return_counts=True)

In [ ]:
# import numpy as np
# from PIL import Image
# # Image.open('data/test/holistic/15999.jpg', mode='r')

In [ ]:
# len(os.listdir('data/test/holistic/'))

In [ ]:
# df_labels = pd.read_csv('data/stratified/rvl-cdip/labels/train.csv',names=['fn','l'],sep=' ')

In [ ]:
# filenames = df_labels[df_labels['l']==12]['fn'].values

In [ ]:
# idx = np.random.randint(0,5000,64)
# images = []
# for f in filenames[idx]:
#     images.append(np.array(Image.open(os.path.join('data/stratified/rvl-cdip/images/',f))))

In [ ]:
# plot_images(images)